In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
# 自身のColab環境によってディレクトリを変更して下さい
import os
os.chdir("/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/")
!pip install transformers
!pip install colorama
!pip install sentencepiece
!pip install mojimoji
!pip install fugashi
!pip install unidic_lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47.4 MB 1.4 MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658837 sha256=7830a1f7fa8e3837058654eedc4f0f4f2440c55c7cb9d82c4b69e167199724b6
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [3]:
# ROOT = "../../cpt-fakenews"
# CUR_DIR = ROOT+"/baseline/code"
# %cd $CUR_DIR

In [4]:
# !pip install GPUtil
# !pip install transformers==4.12.2
# !pip install fugashi
# !pip install mecab-python3
# !pip install ipadic

# !pip install colorama

In [6]:
import os
import gc
import copy
import time
import random
import string
import glob
import datetime

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss, accuracy_score

# For Transformer Models
from transformers import AutoConfig, AutoTokenizer, AutoModel, AdamW, T5Tokenizer

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import pickle
import re
import unicodedata


import regex
import scipy as sp
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, AdamW

import sys
sys.path.append("/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/roberta_japanese")
from tokenization_roberta_japanese import RobertaJapaneseTokenizer

warnings.filterwarnings('ignore')

In [7]:
CONFIG = {"seed": 2022,
          "model_name": r'rinna/japanese-roberta-base',  # r'cl-tohoku/bert-base-japanese-v2'
          "output_dir": "../ex18_japanese_roberta_base_max_len384_7folds/",
          "train_batch_size": 32,
          "test_batch_size": 64,
          "max_length": 100,
          "num_classes": 2,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "dropout": 0.2,
          }

# CONFIG2 = {"seed": 2022,
#           "model_name": r'cl-tohoku/roberta-base-japanese',  # r'cl-tohoku/bert-base-japanese-v2'
#           "output_dir": "../ex19_roberta_japanese_base_max_len384_7folds/",
#           "train_batch_size": 32,
#           "test_batch_size": 64,
#           "max_length": 384,
#           "num_classes": 2,
#           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#           "dropout": 0.2,
#           }

CONFIG2 = {"seed": 2022,
          "model_name": r'cl-tohoku/roberta-base-japanese',  # r'cl-tohoku/bert-base-japanese-v2'
          "output_dir": "../ex23_roberta_japanese_base_max_len384_7folds_rs42/",
          "train_batch_size": 32,
          "test_batch_size": 64,
          "max_length": 100,
          "num_classes": 2,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "dropout": 0.2,
          }

CONFIG3 = {"seed": 2022,
          "model_name": r'microsoft/mdeberta-v3-base',  # r'cl-tohoku/bert-base-japanese-v2'
          "output_dir": "../ex33_mdeberta_v3_base__max_len384_7folds_rs42/",
          "train_batch_size": 32,
          "test_batch_size": 64,
          "max_length": 100,
          "num_classes": 2,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "dropout": 0.2,
          }

CONFIGS = [
    CONFIG,
    CONFIG2,
    CONFIG3,
]

weights = [0.5, 0.5, 0.5]

In [8]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [9]:
df = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/input/nishika/hate_speech/sample_submission.csv")

In [10]:
def read_oof_df(filepath):
    oof_df = pd.read_csv(filepath).sort_values("id").reset_index(drop=True)
    preds = oof_df[["preds", "preds2"]].values
    return oof_df, preds

In [11]:
# oof_df1, preds1 = read_oof_df(os.path.join(CONFIG["output_dir"], "oof_df.csv"))
# preds1.shape

In [13]:
oof_df1, preds1 = read_oof_df('/content/drive/MyDrive/output/nishika/hate_speech/oof_df.csv')
oof_df2, preds2 = read_oof_df('/content/drive/MyDrive/output/nishika/hate_speech/oof_df.csv')
oof_df3, preds3 = read_oof_df('/content/drive/MyDrive/output/nishika/hate_speech/oof_df.csv')

# predictions = (preds1 + preds2) / 2.0
predictions = preds1 * weights[0] + preds2 * weights[1] + preds3 * weights[2]
predictions = np.argmax(predictions, axis=1)
targets = oof_df1["label"].values
acc = accuracy_score(targets, predictions)
print(acc)

0.96


In [15]:
class FakenewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
    
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
        ids = inputs_text['input_ids']
        mask = inputs_text['attention_mask']
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        }

In [16]:
class FakenewsModel(nn.Module):
    def __init__(self, cfg, config_path=None):
        super(FakenewsModel, self).__init__()
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg["model_name"], output_hidden_states=False)
        else:
            self.config = torch.load(config_path)
            
        self.model = AutoModel.from_pretrained(cfg["model_name"], config=self.config)
        self.drop = nn.Dropout(p=cfg["dropout"])
        self.fc = nn.Linear(self.config.hidden_size, cfg['num_classes'])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        if "deberta" in self.config.model_type:
            pooler_output = out.last_hidden_state[:, 0, :]
        else:
            pooler_output = out.pooler_output
        out = self.drop(pooler_output)
        outputs = self.fc(out)
        outputs = self.sigmoid(outputs)
        
        return outputs.squeeze()

In [17]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        
        PREDS.append(outputs.cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [18]:
def inference(cfg, model_paths, dataloader, device):
    model = FakenewsModel(cfg)
    model.to(cfg['device'])
    
    final_preds = []
    for i, path in enumerate(model_paths):
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

['/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold0.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold1.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold2.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold3.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold4.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold5.bin']


In [51]:
def inference_main():
    all_predictions = []
    for cfg, weight in zip(CONFIGS, weights):
        if cfg['model_name'] == "rinna/japanese-roberta-base":
            cfg["tokenizer"] = T5Tokenizer.from_pretrained(cfg['model_name'])
            cfg["tokenizer"].do_lower_case = True
            print("Vocab size: ", CONFIG["tokenizer"].vocab_size)
        elif cfg['model_name'] == "cl-tohoku/roberta-base-japanese":
            cfg["tokenizer"] = RobertaJapaneseTokenizer.from_pretrained(cfg['model_name'])
        else:
            cfg["tokenizer"] = AutoTokenizer.from_pretrained(cfg['model_name'])
        
        MODEL_PATHS = glob.glob('/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/*.bin')
        MODEL_PATHS.sort()

        print(MODEL_PATHS)

        test_dataset = FakenewsDataset(df, cfg['tokenizer'], max_length=cfg['max_length'])
        test_loader = DataLoader(test_dataset, batch_size=cfg['test_batch_size'],
                                 num_workers=2, shuffle=False, pin_memory=True)
        
        preds = inference(cfg, MODEL_PATHS, test_loader, cfg['device'])
        
        all_predictions.append(preds * weight)
        
        del test_dataset, test_loader, preds
        torch.cuda.empty_cache()
        gc.collect()
    
    all_predictions = np.sum(all_predictions, 0)
    
    return all_predictions

In [54]:
print(all_predictions)
display(df.head())

nan


,id,source,text
0,001026808,news4vip,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰1人プロ...
1,00465ac96,livejupiter,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,news4vip,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,newsplus,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...


In [53]:
all_predictions = inference_main()
df['preds'] = np.argmax(all_predictions, axis=1)
df.head()

Vocab size:  32000
['/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold0.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold1.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold2.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold3.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold4.bin', '/content/drive/MyDrive/ColabNotebooks/nishika/hate_speech/fake_1th/1/models/bert-2022-11-13-16-13/model-fold5.bin']


Some weights of the model checkpoint at rinna/japanese-roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at rinna/japanese-roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to b

RuntimeError: ignored

In [ ]:
df["predisFake"] = np.argmax(all_predictions, axis=1)

In [ ]:
print(df["predisFake"].value_counts())

1    1923
0    1858
Name: predisFake, dtype: int64


In [ ]:
submission["label"] = df["predisFake"]
submission.to_csv(f"submission.csv", index=False)